In [1]:
import pandas as pd 
pd.options.display.max_columns = 500
pd.options.display.max_rows = 100
import altair as alt
alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

In [2]:
import logging 
logging.basicConfig()  # Means logs will print in Jupyter Lab

# Set to DEBUG if you want splink to log the SQL statements it's executing under the hood
logging.getLogger("splink").setLevel(logging.INFO)

In [12]:
from utility_functions.demo_utils import get_spark
spark = get_spark()

22/06/14 11:00:34 WARN SimpleFunctionRegistry: The function jaro_winkler_sim replaced a previously registered function.
22/06/14 11:00:34 WARN SimpleFunctionRegistry: The function dmetaphone replaced a previously registered function.


In [13]:
spark

In [29]:
hospital_accounts = spark.read.csv('./sample/hospital_account_info.csv',header=True,inferSchema=True)
hospital_reimbursement = spark.read.csv('./sample/hospital_reimbursement.csv',header=True,inferSchema=True)

In [33]:
hospital_accounts.show(1,truncate=False,vertical=True)
hospital_reimbursement.show(1,truncate=False,vertical=True)

-RECORD 0--------------------------------------------
 unique_id          | 10605                          
 Facility Name      | SAGE MEMORIAL HOSPITAL         
 Address            | STATE ROUTE 264 SOUTH 191      
 City               | GANADO                         
 State              | AZ                             
 ZIP Code           | 86505                          
 County Name        | APACHE                         
 Phone Number       | (928) 755-4541                 
 Hospital Type      | Critical Access Hospitals      
 Hospital Ownership | Voluntary non-profit - Private 
only showing top 1 row

-RECORD 0-----------------------------------------------------
 Provider_Num              | 839987                           
 Provider Name             | SOUTHEAST ALABAMA MEDICAL CENTER 
 Provider Street Address   | 1108 ROSS CLARK CIRCLE           
 Provider City             | DOTHAN                           
 Provider State            | AL                               
 Pro

In [32]:
hospital_accounts = hospital_accounts.withColumnRenamed("Account_Num","unique_id")

In [23]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [],
    "comparison_columns": [
        {
            "col_name": "Facility Name"
        },
        {
            "col_name": "Address"
        },
        {
            "col_name": "City"
        },
        {
            "col_name": "State"
        }
    ]
}


In [34]:
from splink import Splink

linker = Splink(settings, hospital_accounts, spark)
df_e = linker.get_scored_comparisons()

22/06/14 11:13:18 WARN MemoryStore: Not enough space to cache rdd_177_0 in memory! (computed 1879.6 MiB so far)
22/06/14 11:13:18 WARN BlockManager: Block rdd_177_0 could not be removed as it was not found on disk or in memory
22/06/14 11:13:18 WARN BlockManager: Putting block rdd_177_0 failed
INFO:splink.iterate:Iteration 0 complete                                        
INFO:splink.model:The maximum change in parameters was 0.8997805545637466 for key Address, level 0
INFO:splink.iterate:Iteration 1 complete                                        
INFO:splink.model:The maximum change in parameters was 0.03909884657917312 for key City, level 1
INFO:splink.iterate:Iteration 2 complete                                        
INFO:splink.model:The maximum change in parameters was 0.01241580521015842 for key City, level 1
INFO:splink.iterate:Iteration 3 complete                                        
INFO:splink.model:The maximum change in parameters was 0.005250325827393043 for key Stat

In [ ]:
cols_to_inspect = ["match_probability", "match_weight", "unique_id_l", "unique_id_r", \
                   "Facility Name_l", "Facility Name_r", "Address_l", "Address_r", \
                   "City_l", "City_r", "State_l", "State_r"]

df_e.toPandas()[cols_to_inspect].sort_values(["unique_id_l", "unique_id_r"]).head(5)

22/06/14 11:18:46 WARN MemoryStore: Not enough space to cache rdd_177_0 in memory! (computed 1879.6 MiB so far)
22/06/14 11:18:46 WARN BlockManager: Block rdd_177_0 could not be removed as it was not found on disk or in memory
22/06/14 11:18:46 WARN BlockManager: Putting block rdd_177_0 failed
22/06/14 11:19:36 ERROR Utils: Uncaught exception in thread task-result-getter-1
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$readExternal$1(TaskResult.scala:62)
	at org.apache.spark.scheduler.DirectTaskResult$$Lambda$2545/762095392.apply$mcV$sp(Unknown Source)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1428)
	at org.apache.spark.scheduler.DirectTaskResult.readExternal(TaskResult.scala:60)
	at java.io.ObjectInputStream.readExternalData(ObjectInputStream.java:2236)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2185)
	at java

In [9]:
settings = {
    "link_type": "link_only", 
    "max_iterations": 20,
    "blocking_rules": [
    ],
    "comparison_columns": [
       {
            "custom_name": "name_inversion",
            "custom_columns_used": ["first_name", "surname"],
            "case_expression": sql_case_expression,
            "num_levels": 5
        },
        {
            "col_name": "city",
            "num_levels": 3
        },
        {
            "col_name": "email",
            "num_levels": 3
        },
        {
            "col_name": "dob"
        }
    ],
    "additional_columns_to_retain": ["group"],
    "em_convergence": 0.01,
    "max_iterations": 4,
}

compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
            'Provider Name',
            threshold=0.85,
            label='Hosp_Name')
compare.string('Address',
            'Provider Street Address',
            method='jarowinkler',
            threshold=0.85,
            label='Hosp_Address')

NameError: name 'sql_case_expression' is not defined

In [ ]:
from splink import Splink  
linker = Splink(settings, spark, df=df) 
df_e = linker.get_scored_comparisons()